<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C1-Basic_Networking_Tools/TCP_Proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a TCP Proxy

There are several reasons to have a __TCP__ proxy in your tool belt. You might use one for forwarding traffic to bounce from host to host, or when assessing network based software. When performing penetration tests in enterprise environments, you probably won't be able to run __Wireshark__; nor will you be able to load drivers to sniff the loopback on Windows, and network segmentation will prevent you from running your tools directly against your target host. We've built simple Python proxies, like this one, in various cases to help you understand unknown protocols, modify traffic being sent to an application, and create test cases for fuzzers. 

The proxy has a few moving parts. Let's summarize the four main functions we need to write. We need to display the communication between the local and remote machines to the console (__hexdump__). We need to receive data from an incoming socket from either the local or remote machine (__receive_from__). We need to manage the traffic direction between remote and local machines (__proxy_handler__). Finally, we need to set up a listening socket and pass it to our __proxy_handler__ (__server_loop__).

Let's get to it. Open a new file called __proxy.py__:

In [ ]:
import sys
import socket
import threading

HEX_FILTER = ''.join(
    [len(repr(chr(i))) == 3) and chr(i) or '.' for i in range(256)]) #1

def hexdump(src, length=16, show=True):
    if isinstance(src, bytes): #2
        src = src.decode()

    results = list()
    for i in range(0, len(src), length):
        word = str(src[i:i+length]) #3

        printable = word.translate(HEX_FILTER) #4
        hexa = ' '.join([f'{ord(c):02X}' for c in word])
        hexwidth = length * 3
        results.append(f'{i:04x}  {hexa:<{hexwidth}}  {printable}') #5
    if show:
        for line in results:
            print(line)
    else:
        return results

We start with a few imports. Then we define a __hexdump__ function that takes some input as bytes or a string and prints a __hexdump__ to the console. That is, it will output the packet details with both their hexadecimal values and __ASCII__ printable characters. This is useful for understanding unknown protocols, finding user credentials in plaintext protocols, and much more. We create a __HEX_FILTER__ string __[1]__ that contains __ASCII__ rpintable characters, if one exists, or a dot (.) if such a representation doesn't exist. For an example of what this string could contain, let's look at the character representations of two integers, __30__ and __65__, in an interactive Python shell:
```
>>> chr(65)
'A'
>>> chr(30)
'\x1e'
>>> len(repr(chr(65)))
3
>>> len(repr(chr(30)))
6
```

The character representation of __65__ is printable and the character representation of __30__ is not. As you can see, the representation of the printable character ha s a length of __3__. We use that fact to create the final __HEX_FILTER__ string: Provide the chjaracter if possible and a dot (.) if not.

The list comprehension used to create the string employs a __Boolean short circuit__ technique, which sounds pretty fancy. Let's break it down: For each integer in the range of __0__ to __255__, if the length of the corresponding character equals __3__, we get the character (chr(i)). Otherwise, we get a dot (.). Then we __join__ that list into a string so it looks something like this:
```
'............................... !"#&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[.]^_`abcdefghijklmnopqrstuvwxyz{|}~.................................. OTHER WEIRD CHARACTERS FOR YOU MAH NIGGAS'
```

The list comprehension gives a printable character representation of the first 256 integers. Now we can create the __hexdump__ function. First, we make sure we have a string, decoding the bytes if a byte string was passed in __[2]__. Then we grab a piece of the string to dump and put it into the __word__ variable __[3]__. We use the __translate__ built-in function to substitute the string representation of each character for the corresponding character in the raw string (__printable__) __[4]__. Likewise, we substitute the hex representation of the integer value of every chacater in the raw string (__hexa__). Finally, we create a new array to hold the strings, __result__, that contains the hex value of the index of the first byte in the word, the hex value of the word, and its printable representation __[5]__. The ouput looks like this:
```
>>> hexdump('python rocks\n and proxies roll\n')
0000 70 79 74 68 6F 6E 20 72 6F 63 6B 73 0A 20 61 6E python rocks. an
0010 64 20 70 72 6F 78 69 65 73 20 72 6F 6C 6C 0A    d proxies roll.
```

This function provides us with a way to watch the communication going throught the proxy in real time. Now let's create a function that the two ends of the proxy will use to receive data:

In [ ]:
def receive_from(connection):
    buffer = b""
    connection.settimeout(5)
    try:
        while True:
            data = connection.recv(4096)
            if not data:
                break
            buffer += data
    except Exception as e:
        pass
    return buffer

For receiving both local and remote data, we pass in the socket object to be used. We create an empty byte string, __buffer__, that will accumulate responses from the socket __[1]__. By default, we set a five second time-out, which might be aggressive if you're proxying traffic to other countries or over lossy networks, so increase the time-out as necessary. We set up a loop to read response data into the __buffer__ __[2]__ until there's no more data or we time out. Finally, we return the __buffer__ byte string to the caller, which could be either the local or remote machine.

Sometimes you may want to modify the response or request packets before the proxy sends them on their way. Let's add a couple of functions (__request_hadler__ and __response_handler__) to do just that:

In [ ]:
def request_handler(buffer):
    # perform packet modifications
    return buffer

def response_handler(buffer):
    # perform packet modifications
    return buffer

Inside these functions, you can modify the packet contents, perform fuzzing tasks, test for authentication issues, or do whatever else your heart desires. This can be useful, for example, if you find plain-text user credentials being sent and want to try to elevate privileges on an application by passing in __admin__ instead of your own username.

Let's dive into the __proxy_handler__ function now by adding this code:

In [ ]:
def proxy_handler(client_socket, remote_host, remote_port, receive_first):
    remote_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    remote_socket.connect((remote_host, remote_port)) #1

    if receive_first: #2
        remote_buffer = receive_from(remote_socket)
        hexdump(remove_buffer)

    remote_buffer = response_handler(remote_buffer) #3
    if len(remote_buffer):
        print("[<==] Sending %d bytes to localhost." % len(remote_buffer))
        client_socket.send(remote_buffer)

    while True:
        local_buffer = receive_from(client_socket)
        if len(local_buffer):
            line = "[==>] Received %d bytes from localhost." % len(local_buffer)
            print(line)
            hexdump(local_buffer)

            local_buffer = request_handler(local_buffer)
            remote_socket.send(local_buffer)
            print("[==>] Sent to remote.")

        remote_buffer = receive_from(remote_socket)
        if len(remote_buffer):
            print("[<==] Received %d byte from remote." % len(remote_buffer))
            hexdump(remote_buffer)

            remote_buffer = response_handler(remote_buffer)
            client_socket.send(remote_buffer)
            print("[<==] Sent to localhost.")

        if not len(local_buffer) or not len(remote_buffer): #4
            client_socket.close()
            remote_socket.close()
            print("[*] No more data. Closing connections.")
            break

This function contains the bulk of the logic for our proxy. To start off, we connect to the remote host __[1]__. Then we check to make sure we don't need to first initiate a connection to the remote side and request data before going inot the main loop __[2]__. Some server daemons will expect you yo do this (__FTP__ servers typically send a banner first, for example). We then use the __receive_from__ function for both sides of the communication. It accepts